In [8]:
# pip install pyconll

In [9]:
## ToDo
## Mention in ReadME
    ## Run this script iteratively to ensure prerequisite of number of columns for some checks 

In [10]:
## ToDo Done
## NestedTag value must be True
## Coref value must be Int
## Value in quotation should be part of token
## Value in misc should not have backslash 
## Value should either be Matrix_Tag, True or string 

In [11]:
## ToDO ##Done
## Head should be integer
## Ensure single punctuation
## Head of root should be 0

In [12]:
## Validate lemmma and form ##Done
## If not devnagari and int it should be from predefined list ['pause',..]

In [13]:
## ToDo
## More than one occurence of deprel and if they have same head, then
## check whether second occurence should have coref of token of first occurence of deprel

In [40]:
## ToDo
## Head should not be token id of same token

In [1]:
import os
import re
import pyconll 

In [2]:
filepath = "../../parse_gold/" ## path where gold data is stored in phase wise manner

In [3]:
dirs = sorted(os.listdir(filepath)) 

In [4]:
# validate_columns(text)

In [5]:
def find_whitespace(line):
    pattern = re.compile(r'(?!\t)(?!\n)\s+')
    matches = pattern.finditer(line)
    results = []
    for match in matches:
        start = match.start()
        
        whitespace = match.group()
        context = line[max(0, start - 4):start + 4]
        results.append((whitespace, context))
    return results

In [6]:
def find_whitespace_meta_data(line):
    pattern = re.compile(r'(?!\t)(?!\n)\s{2,}')
    matches = pattern.finditer(line)
    results = []
    for match in matches:
        start = match.start()
        
        whitespace = match.group()
        context = line[max(0, start - 4):start + 4]
        results.append((whitespace, context))
    return results

In [7]:
def validate_whitespaces(output_file_path):
    sent_id = 0
    list_failed_validate_whitespaces = []
    list_failed_validate_whitespaces_meta = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
                        
            if "sent_id" in line:
                sent_id += 1
                
            if line.startswith("#") or line.startswith("\n"):
                res = find_whitespace_meta_data(line)
                if len(res)!=0:
                    list_failed_validate_whitespaces_meta.append([(f"Sentence Id: {sent_id}"),(f"More than one whitespaces: {res}")])

                continue
            
            # whitespaces = re.findall("\s", line)
            whitespaces = find_whitespace(line)
            if len(whitespaces)!=0:
                token_id = re.findall(r'\d+', line)[0]
                list_failed_validate_whitespaces.append([(f"Sentence Id: {sent_id}"), (f"Token Id: {token_id}"), (f"Whitespaces: {whitespaces}")])
    
    return list_failed_validate_whitespaces, list_failed_validate_whitespaces_meta

In [8]:
def validate_newline(output_file_path):
    sent_id = 0
    list_failed_validate_newline = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                sent_id += 1
            if line.startswith("#") or line.startswith("\n"):
                continue
            if '\n' not in line:
                token_id = re.findall(r'\d+', line)[0]
                list_failed_validate_newline.append([(f"Sentence Id: {sent_id}"), (f"Token Id: {token_id}"), ("No new line character is present")])
        
    return list_failed_validate_newline

In [9]:
def validate_columns(line):
    # import pdb;pdb.set_trace()
    cols = (line.split("\t"))
    col_count = len(cols)
    tabs = re.findall(r'\t{2,}', line)  ## Look for more than one consecutive tabs
    if col_count != 10:
        return f"Number of columns: {col_count}"
    elif len(tabs)!=0:
        return "Empty column found"
    return False

In [10]:
def validate_num_cols(output_file_path):
    
    # file = os.path.basename(output_file_path)
    sent_id = 0
    list_failed_validate_num_cols = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                sent_id += 1
            if line.startswith("#") or line.startswith("\n"):
                continue
            # print(line)
            col_count = validate_columns(line)
            if col_count:
                token_id = re.findall(r'\d+', line)[0]
                # print(line[0])
                # print(file, sent_id, col_count)
                list_failed_validate_num_cols.append([(f"Sentence Id: {sent_id}"), (f"Token Id: {token_id}"), (col_count)])
   
        
        
    return list_failed_validate_num_cols
                    

In [11]:
def validate_undescores(output_file_path):
    sent_id = 0
    list_underscores = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
                
            underscores = re.findall(r'\_{2,}', line)
            if len(underscores)!=0:
                list_underscores.append([(f"Sentence Id in the file: {sent_id_in_file}"), (f"More that one underscores found in {line}")])
                
    return list_underscores

In [12]:
def validate_sent_token_id(output_file_path):
    sent_id = 0
    token_id = 0
    list_failed_validate_sent_id = []
    list_failed_validate_token_id = []
    list_failed_validate_no_token_id = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                if token_id>0:
                    last_token_prev_sent = token_id
                    # import pdb;pdb.set_trace()
                    if last_token_prev_sent!=total_tokens_meta:
                        list_failed_validate_no_token_id.append([(f"Sentence Id in the file: {sent_id_in_file+1}"), "Mismatch between no of tokens in meta sentence and token ids"])
                token_id = 0
                sent_id += 1
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
                if sent_id != sent_id_in_file:
                    list_failed_validate_sent_id.append([(f"Required Sentence Id: {sent_id}"), (f"Sentence Id in the file: {sent_id_in_file}")])
                # list_failed_validate_token_id.append("$"*100)
            
            if "Sentence" in line:
                sent_string = line.split("=")[1]
                tokens = sent_string.split(" ")
                # print(tokens)
                total_tokens_meta = len(tokens)-1
                # print(total_tokens_meta)
            
            

            if line.startswith("#") or line.startswith("\n"):
                continue
                
            token_id +=1
            token_id_in_file = int(re.findall(r'\d+', line)[0])
            if token_id != token_id_in_file:
                list_failed_validate_token_id.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Required Token Id: {token_id}"), (f"Token Id in the file: {token_id_in_file}")])
            
            
            
            # if total_tokens_meta
                
                # import pdb;pdb.set_trace()
    return list_failed_validate_sent_id, list_failed_validate_token_id, list_failed_validate_no_token_id
                

In [13]:
list_attributes = []

In [14]:
import re
re.findall(r'[\u0900-\u097F]+', "ओए तुम dfg सुन रहे ना")

['ओए', 'तुम', 'सुन', 'रहे', 'ना']

In [15]:
def validate_lemma_form(output_file_path, list_attributes):
    sent_id = 0
    token_id = 0
    list_failed_upos = []
    list_failed_deprel = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                token_id = 0
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):
                form = line.split("\t")[1]
                lemma = line.split("\t")[2]
                
                devanagari_words = re.findall(r'[\u0900-\u097F]+', form)
                

In [16]:
list_deprel = '''acl
acl:relcl
advcl
advmod
amod
aux
aux:pass
case
cc
compound
conj
cop
dep
det
dislocated
iobj
mark
nmod
nsubj
nsubj:pass
nummod
obj
obl
punct
reparandum
root
vocative
xcomp'''.split("\n")

In [17]:
len(list_deprel)

28

In [18]:
list_upos = '''ADJ
ADP
ADV
AUX
CCONJ
DET
INTJ
NOUN
NUM
PART
PRON
PROPN
PUNCT
SCONJ
VERB
X'''.split("\n")

In [19]:
len(list_upos)

16

In [20]:
def validate_upos_deprel(output_file_path, list_upos, list_deprel):
    sent_id = 0
    token_id = 0
    list_failed_upos = []
    list_failed_deprel = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                token_id = 0
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):
                upos = line.split("\t")[3]
                deprel = line.split("\t")[7]
                if upos not in list_upos:
                    list_failed_upos.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Required Token Id: {token_id}"), (f"Incorrect upos found : {upos}")])
            
                colons = re.findall(r'\:', deprel)  ## can be removed
                # print(deprel)
                # import pdb;pdb.set_trace()
                count_colon = len(colons)
                if (deprel not in list_deprel) or (count_colon>1):
                    list_failed_deprel.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Required Token Id: {token_id}"), (f"Incorrect deprel found : {deprel}")])
                # break
    return list_failed_upos, list_failed_deprel

In [21]:
def validate_root_punct(output_file_path):
    ## ToDO
    ## Head should be integer
    ## Ensure single punctuation
    ## Head of root should be 0
    sent_id = 0
    token_id = 0
    root_token_id = 0
    list_root = []
    list_punct = []
    list_failed_root = []
    list_failed_punct = []
    list_failed_head = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                list_root = []
                list_punct = []
                token_id = 0
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):  ## Case when columns are equal to 10
                upos = line.split("\t")[3]
                deprel = line.split("\t")[7]
                head = int(line.split("\t")[6])
                
                if not isinstance(head, int):
                    list_failed_head.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Head is not an integer")])
                
                if deprel == 'root':
                    root_head = int(line.split("\t")[6])
                    root_token_id = token_id
                    list_root.append(deprel)
                    if root_head !=0:
                        list_failed_root.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Wrong head of root: {root_head}")])

                if len(list_root)>1:
                    list_failed_root.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"More than one root found")])
                
                if upos == 'PUNCT':
                    # import pdb;pdb.set_trace()
                    head = int(line.split("\t")[6])
                    list_punct.append(upos)
                    if head!=root_token_id:
                        list_failed_punct.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Head of PUNCT is {head} and not root token_id {root_token_id}")])
                    
                    if len(list_punct)>1:
                        list_failed_punct.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"More than one punctuation upos found")])

    return list_failed_root, list_failed_punct, list_failed_head

In [22]:
list_tags = ['Repair',  ## To be cross verify with data
 'CodeSwitch',
 'Disfluency',
 'Hesitation',
 'Expletive',
 'Quote',
 'NestedTag',
 'Coref']

In [23]:
def validate_misc(output_file_path, list_tags):
    ## ToDo
    ## NestedTag value must be True
    ## Coref value must be Int
    ## Value in quotation should be part of token
    ## Value in misc should not have backslash 
    ## Value should either be Matrix_Tag or string 
    sent_id = 0
    token_id = 0
    list_root = []
    list_failed_misc = []
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                list_root = []
                token_id = 0
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):  ## Case when columns are equal to 10
                misc = line.split("\t")[9]
                misc = misc.strip('\n')
                if misc != "_":
                    if "|" in misc:
                        misc_list = misc.split("|")
                        for misc_attributes in misc_list:
                            key = misc_attributes.split("=")[0]
                            value = misc_attributes.split("=")[1]
                            
                            if key not in list_tags:
                                list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect annotation found in misc {key}")])
                            # elif isinstance(value, str):
                                
                            elif key=="NestedTag" and value!="True":
                                list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect annotation found in misc {key, value}")])
                            
                            elif key =="Coref" and isinstance(int(value), int):
                                list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect value found in misc {value}")])
                            elif "\\" in value:  
                                list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect value found in misc {value}")])
                        
                                
                    else:
                        misc_list = misc.split("=")
                        key = misc_list[0]
                        value = misc_list[1]
                        
                        if key not in list_tags:
                            list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect annotation found in misc {key}")])
                        elif key=="NestedTag" and value!="True":
                            list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect annotation found in misc {key, value}")])
                        elif key =="Coref" and isinstance(int(value), int):
                            list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect value found in misc {value}")])
                        elif "\\" in value:  
                            list_failed_misc.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect value found in misc {value}")])

                            
                            
    return list_failed_misc
                

In [24]:

# import counter class from collections module
from collections import Counter
Counter([(3,'abc'), (1,'abc') , (1,'abc')])

Counter({(3, 'abc'): 1, (1, 'abc'): 2})

In [25]:
## Validate lemmma and form
## If not devnagari and int it should be from predefined list ['pause',..]

In [26]:
# re.findall("^(?!.*[\u0900-\u097F])(?!^\d+$)\w+$", "laughter")

In [27]:
# import re

# pattern = r"[\u0900-\u097F]|\d+|[pause]|[aside]|[b_aside]|[e_aside]|[laughter]|[noise]|[incomprehensible]"
# text = "[pause]"

# matches = re.findall(pattern, text)
# print(matches)

In [28]:
# att_list = ["[pause]", "[aside]", "[b_aside]", "[e_aside]", "[laughter]", "[noise]", "[incomprehensible]"]
# check_form_lemma( "11", att_list)

In [30]:
def check_form_lemma(string, attribute_list):
 
    if string not in attribute_list: 
        if "_" not in string:
            devanagari_pattern = r'[\u0900-\u097F]'
            devnagari_pattern = f'{devanagari_pattern}'
            devnagari = re.findall(devnagari_pattern, string)
            flag_devnagari = len(devnagari) == len(string)

            integer_pattern = r'\d+'
            digit_str = re.findall(integer_pattern, string)
            # import pdb;pdb.set_trace()
            try:
                digit_int = int(digit_str[0])
                flag_integer = isinstance(digit_int, int)
            except:
                flag_integer = False
                
            # flag = flag_devnagari or flag_integer
        else:
            string_list = string.split("_")
            flag = False
            for string in string_list:
                flag = flag or check_form_lemma(string, attribute_list)
                
    elif string in attribute_list:
        flag = True
    else:
        flag = False
    
    return flag

In [31]:
def validate_form_lemma(output_file_path):
    sent_id = 0
    token_id = 0
    list_failed_form = []
    list_failed_lemma = []
    attribute_list = ["[pause]", "[aside]", "[b_aside]", "[e_aside]", "[laughter]", "[noise]", "[incomprehensible]", "।", "[anonymized]"]
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                token_id = 0
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):  ## Case when columns are equal to 10
                form = line.split("\t")[1]
                lemma = line.split("\t")[2]
        
                
                if not check_form_lemma(form, attribute_list):
                    list_failed_form.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect form found {form}")])
                if not check_form_lemma(lemma, attribute_list):
                    list_failed_lemma.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect form found {lemma}")])
    
    return list_failed_form, list_failed_lemma

In [32]:
def validate_subj_obj(output_file_path):
    ## ToDo
    ## More than one occurence of deprel and if they have same head, then
    ## check whether second occurence should have coref of token of first occurence of deprel
    sent_id = 0
    token_id = 0
    list_failed_subj_obj = []
    deprel_list = []
    deprel_dict = {}
    deprel_check = ['nsubj', 'obj', 'iobj']
    with open(output_file_path, 'r') as output_file:
        for line in output_file:
            if "sent_id" in line:
                token_id = 0
                deprel_list = []
                deprel_dict = {}
                sent_id_in_file = int(re.findall(r'\d+', line)[0])
            
            if line.startswith("#") or line.startswith("\n"):
                continue
            
            token_id += 1
            if not (validate_columns(line)):  ## Case when columns are equal to 10
                head = line.split("\t")[6]
                deprel = line.split("\t")[7]
                misc = line.split("\t")[9]
                misc = misc.replace("\n", "")
                
                if deprel in deprel_check:
                    # for i in deprel:
                    deprel_list.append((head, deprel))
                    deprel_dict.update({(head, deprel):token_id})
                    deprel_count = Counter(deprel_list)     
                    print(deprel_count)
                    for _, count in deprel_count.items():
                        print(count)
                        if count>1:
                            # import pdb;pdb.set_trace()
                            if "_" in misc:
                                print("#############")
                                list_failed_subj_obj.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect Coref in misc: {misc}")])
                            else:
                                misc_list = misc.split("=")
                                
                                print(misc_list, sent_id_in_file,token_id, output_file_path)
                                coref_file = misc_list[1]
                                req_coref = deprel_dict[(head,deprel)]

                                if int(coref_file)!=int(req_coref):
                                    list_failed_subj_obj.append([(f"Sentence Id in the file: {sent_id_in_file}"),(f"Token Id: {token_id}"), (f"Incorrect Coref in misc: {misc}")])
                            count =0
                            # deprel_to_check = deprel

                
                
    return list_failed_subj_obj

In [34]:
sanity_checks = []

for phase in dirs:
    if not os.path.isfile(os.path.join(filepath, phase)):    
        phase_path = os.path.join(filepath, phase)
        output_files = sorted(os.listdir(phase_path))
        # sanity_checks['phase'] = phase
        for file in output_files:
            output_file_path = os.path.join(phase_path, file)
            if os.path.isfile(output_file_path):
                        
                        
                # validate number of columns
                sanity_check_cols = validate_num_cols(output_file_path)
                # print(sanity_check_cols)
                if len(sanity_check_cols)!=0:
                    sanity_checks.append([phase, file, 'validate_num_cols',  sanity_check_cols])
            
                
                ## Validate whitespaces
                sanity_check_whitespaces,sanity_check_whitespaces_meta = validate_whitespaces(output_file_path)
                if len(sanity_check_whitespaces)!=0:
                    sanity_checks.append([phase, file, 'validate_whitespaces', sanity_check_whitespaces])
                if len(sanity_check_whitespaces_meta)!=0:
                    sanity_checks.append([phase, file, 'meta_data_validate_whitespaces', sanity_check_whitespaces_meta])
                
                
                ## Validate newline character at every row
#                 sanity_check_newline_char = validate_newline(output_file_path)
#                 if len(sanity_check_newline_char)!=0:
#                     sanity_checks.append([phase, file, 'validate_newline_char', sanity_check_newline_char])
                    
                    
                ## validate sequence of sentence id and token id
                sanity_check_sent_id, sanity_check_token_id, sanity_check_num_token_id = validate_sent_token_id(output_file_path)
                if len(sanity_check_sent_id)!=0:
                    sanity_checks.append([phase, file, 'validate_sent_id_sequence', sanity_check_sent_id])
                if len(sanity_check_token_id)!=0:
                    sanity_checks.append([phase, file, 'validate_token_id_sequence', sanity_check_token_id])
                # print(sanity_check_num_token_id)
                if len(sanity_check_num_token_id)!=0:
                    sanity_checks.append([phase, file, 'validate_num_token_id_sequence', sanity_check_num_token_id])
                                
                ## validate double underscores
                sanity_check_list_underscores = validate_undescores(output_file_path)
                # print(sanity_check_list_underscores)
                if len(sanity_check_list_underscores)!=0:
                    sanity_checks.append([phase, file, 'validate_underscores', sanity_check_list_underscores])


                    
                ## validate annotations tags
                # sanity_check_annotation_tags = validate_annotation_tags(output_file_path)
                # if len(sanity_check_annotation_tags)!=0:
                
                 # Validate form and lemma
                # sanity_check_form,sanity_check_lemma  = validate_form_lemma(output_file_path)
                # if len(sanity_check_form)!=0:
                #     sanity_checks.append([phase, file, 'validate_form', sanity_check_form])
                # if len(sanity_check_lemma)!=0:
                #     sanity_checks.append([phase, file, 'validate_lemma', sanity_check_lemma])
                
        
            
                ## Validate deprel
                sanity_check_upos, sanity_check_deprel = validate_upos_deprel(output_file_path, list_upos, list_deprel)
                if len(sanity_check_upos)!=0:
                    sanity_checks.append([phase, file, 'validate_upos', sanity_check_upos])
                if len(sanity_check_deprel)!=0:
                    sanity_checks.append([phase, file, 'validate_deprels', sanity_check_deprel])
                    
                ## Validate root and punctuation
                sanity_check_root, sanity_check_punct, sanity_check_head = validate_root_punct(output_file_path)
                if len(sanity_check_root)!=0:
                    sanity_checks.append([phase, file, 'validate_root', sanity_check_root])
                if len(sanity_check_punct)!=0:
                    sanity_checks.append([phase, file, 'validate_punct', sanity_check_punct])
                if len(sanity_check_head)!=0:
                    sanity_checks.append([phase, file, 'validate_head', sanity_check_head])
                    
                    
                    
                ## Validate Misc
                sanity_check_misc = validate_misc(output_file_path, list_tags)
                if len(sanity_check_misc)!=0:
                    sanity_checks.append([phase, file, 'validate_misc', sanity_check_misc])
                    
                ## Validate subj/obj
                # sanity_check_coref = validate_subj_obj(output_file_path)
                # if len(sanity_check_coref)!=0:
                #     sanity_checks.append([phase, file, 'validate_coref', sanity_check_coref])
                
               
            

In [35]:
sanity_checks

[['Phase1',
  'hi_1385_gold.conllu',
  'validate_num_cols',
  [['Sentence Id: 5', 'Token Id: 6', 'Empty column found']]],
 ['Phase1',
  'hi_4370_gold.conllu',
  'validate_misc',
  [['Sentence Id in the file: 121',
    'Token Id: 8',
    'Incorrect value found in misc 3'],
   ['Sentence Id in the file: 284',
    'Token Id: 11',
    'Incorrect value found in misc 8'],
   ['Sentence Id in the file: 292',
    'Token Id: 17',
    'Incorrect value found in misc 12']]],
 ['Phase1',
  'hi_6322_gold.conllu',
  'validate_punct',
  [['Sentence Id in the file: 103',
    'Token Id: 14',
    'Head of PUNCT is 12 and not root token_id 5'],
   ['Sentence Id in the file: 198',
    'Token Id: 12',
    'Head of PUNCT is 8 and not root token_id 2'],
   ['Sentence Id in the file: 440',
    'Token Id: 9',
    'Head of PUNCT is 6 and not root token_id 3'],
   ['Sentence Id in the file: 622',
    'Token Id: 7',
    'Head of PUNCT is 4 and not root token_id 1']]],
 ['Phase1',
  'hi_6322_gold.conllu',
  'valida

In [36]:
import pandas as pd

In [37]:
df = pd.DataFrame(sanity_checks)

In [38]:
df = df.explode(3, ignore_index=True)

In [39]:
df.to_csv("sanity_checks_v2.csv", index=False)

In [194]:
sanity

{'sanity_check': 'validate_num_cols',
 'phase': 'Phase1',
 'file': 'hi_6523_gold.conllu',
 'details': [[('Sentence Id: ', 450),
   ('Token Id: ', '3'),
   ('Number of columns: ', 9)]]}

## White spaces

In [70]:
invalid_files = []
for phase in dirs:
    if not os.path.isfile(os.path.join(filepath, phase)):    
        # print(phase)
        phase_path = os.path.join(filepath, phase)
        output_files = sorted(os.listdir(phase_path))
        # print(filepath, os.path.join(filepath, phase),output_files)
        
        for file in output_files:
            output_file_path = os.path.join(phase_path, file)
            print(output_file_path)
            try:
                output_file = pyconll.load_from_file(output_file_path)
            except pyconll.exception.ParseError as e:
                print(e)
                invalid_files.append(output_file_path)
#                 with open(output_file_path, 'r') as output_file:
#                     for line in output_file:
#                         # print(line)
#                         if not line.startswith("#") or line.strip() == "":
                            
#                             # if " " or "  " in line:
#                             #     print(f"Whitespace found at line {line} in {file}")
                            
#                             count_cols = len(line.split('\t'))
#                             if count_cols!=10:
#                                 # import pdb;pdb.set_trace()
#                                 print(f"Invalid number of columns at line {line} in {file}: {(count_cols)} columns found.")
                                
            
            for sentence in output_file:
                
                for token in sentence:
                    # print(token.startswith("#"))
                    # if token.line.startswith('#'):
                    print(token.conll)
                    # print(re.match(r"^\s+$", token.form))
            
            
                break

../../parse_gold/Phase1/hi_1385_gold.conllu
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a69b640>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a69b240>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f314ba5ab40>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f314ba5a9c0>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f314ba5acc0>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f314ba5aa40>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f314ba5ac40>>
../../parse_gold/Phase1/hi_4370_gold.conllu
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a9b8bc0>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a9b8f40>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a9b86c0>>
<bound method Token.conll of <pyconll.unit.token.Token object at 0x7f315a9b8c40>>
../../pars

In [ ]:
file_path

In [8]:
path = "../../DC_ph1_gold/hi_1385_gold.conllu"

In [9]:
file_1385 = pyconll.load_from_file(path)

In [16]:
for sentence in file_1385:
    sentence_string = sentence.meta_value("Sentence")
    list_sent = sentence_string.split(" ")
    token_count = len(list_sent)
    print("#########")
    print(token_count)
    
    for token in sentence:
        print(token.id)

#########
7
1
2
3
4
5
6
7
#########
5
1
2
3
4
5
#########
13
1
2
3
4
5
6
7
8
9
10
11
12
13
#########
3
1
2
3
#########
8
1
2
3
4
5
6
7
8
#########
8
1
2
3
4
5
6
7
8
#########
2
1
2
#########
5
1
2
3
4
5
#########
2
1
2
#########
5
1
2
3
4
5
#########
2
1
2
#########
2
1
2
#########
2
1
2
#########
3
1
2
3
#########
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
#########
2
1
2
#########
5
1
2
3
4
5
#########
7
1
2
3
4
5
6
7
#########
3
1
2
3
#########
5
1
2
3
4
5
#########
6
1
2
3
4
5
6
#########
3
1
2
3
#########
2
1
2
#########
7
1
2
3
4
5
6
7
#########
9
1
2
3
4
5
6
7
8
9
#########
10
1
2
3
4
5
6
7
8
9
10
#########
9
1
2
3
4
5
6
7
8
9
#########
7
1
2
3
4
5
6
7
#########
6
1
2
3
4
5
6
#########
2
1
2
#########
3
1
2
3
#########
2
1
2
#########
11
1
2
3
4
5
6
7
8
9
10
11
#########
2
1
2
#########
5
1
2
3
4
5
#########
11
1
2
3
4
5
6
7
8
9
10
11
#########
2
1
2
#########
10
1
2
3
4
5
6
7
8
9
10
#########
8
1
2
3
4
5
6
7
8
#########
8
1
2
3
4
5
6
7
8
#########
6
1
2
3
4
5
6
#########
6